In [ ]:
#Visual display of just exterior buffer
import os
import random
from PIL import Image, ImageDraw

def visualize_bounding_boxes(image_path, label_path):
    # Load the image
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)

    # Read the label file
    with open(label_path, 'r') as f:
        lines = f.readlines()

    # Draw bounding boxes on the image
    for line in lines:
        class_id, x, y, width, height = map(float, line.strip().split(' '))
        left = (x - width / 2) * image.width
        top = (y - height / 2) * image.height
        right = (x + width / 2) * image.width
        bottom = (y + height / 2) * image.height
        draw.rectangle([(left, top), (right, bottom)], outline='red')

    # Show the image with labels
    image.show()

def create_neighbor_8_buffer(image_path, label_path, output_path, buffer_size=5):
    # Load the image
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)

    # Read the label file
    with open(label_path, 'r') as f:
        lines = f.readlines()

    # Draw bounding boxes and buffer
    for line in lines:
        class_id, x, y, width, height = map(float, line.strip().split(' '))
        left = (x - width / 2) * image.width
        top = (y - height / 2) * image.height
        right = (x + width / 2) * image.width
        bottom = (y + height / 2) * image.height

        # Draw the original bounding box in red
        draw.rectangle([(left, top), (right, bottom)], outline='red')

        # Create a buffer around the bounding box
        left_buffered = max(0, left - buffer_size)
        top_buffered = max(0, top - buffer_size)
        right_buffered = min(image.width, right + buffer_size)
        bottom_buffered = min(image.height, bottom + buffer_size)

        # Draw the buffer area in blue
        draw.rectangle([(left_buffered, top_buffered), (right_buffered, bottom_buffered)], outline='blue')

    # Save the image with bounding boxes and buffer
    image.save(output_path)
    image.show()

# Example usage:
images_directory = 'E:\\sample\\img'
labels_directory = 'E:\\sample\\label'

image_files = os.listdir(images_directory)
random_image_file = random.choice(image_files)
image_path = os.path.join(images_directory, random_image_file)

label_file = os.path.splitext(random_image_file)[0] + '.txt'
label_path = os.path.join(labels_directory, label_file)

# Visualize bounding boxes
visualize_bounding_boxes(image_path, label_path)

# Create and visualize the neighbor-8 buffer
output_buffered_image_path = 'E:\\sample\\buffer.jpg'
create_neighbor_8_buffer(image_path, label_path, output_buffered_image_path, buffer_size=5)


In [ ]:
#Visual display of both interior and exterior buffers
import os
import random
from PIL import Image, ImageDraw

def visualize_bounding_boxes(image_path, label_path):
    # Load the image
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)

    # Read the label file
    with open(label_path, 'r') as f:
        lines = f.readlines()

    # Draw bounding boxes on the image
    for line in lines:
        class_id, x, y, width, height = map(float, line.strip().split(' '))
        left = (x - width / 2) * image.width
        top = (y - height / 2) * image.height
        right = (x + width / 2) * image.width
        bottom = (y + height / 2) * image.height
        draw.rectangle([(left, top), (right, bottom)], outline='red')

    # Show the image with labels
    image.show()

def create_neighbor_8_buffers(image_path, label_path, output_path, exterior_buffer_size=5, interior_buffer_size=2):
    # Load the image
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)

    # Read the label file
    with open(label_path, 'r') as f:
        lines = f.readlines()

    # Draw bounding boxes, exterior buffer, and interior buffer
    for line in lines:
        class_id, x, y, width, height = map(float, line.strip().split(' '))
        left = (x - width / 2) * image.width
        top = (y - height / 2) * image.height
        right = (x + width / 2) * image.width
        bottom = (y + height / 2) * image.height

        # Draw the original bounding box in red
        draw.rectangle([(left, top), (right, bottom)], outline='red')

        # Create an exterior buffer around the bounding box in blue
        left_exterior_buffer = max(0, left - exterior_buffer_size)
        top_exterior_buffer = max(0, top - exterior_buffer_size)
        right_exterior_buffer = min(image.width, right + exterior_buffer_size)
        bottom_exterior_buffer = min(image.height, bottom + exterior_buffer_size)
        draw.rectangle([(left_exterior_buffer, top_exterior_buffer), (right_exterior_buffer, bottom_exterior_buffer)], outline='blue')

        # Create an interior buffer around the bounding box in green
        left_interior_buffer = max(0, left + interior_buffer_size)
        top_interior_buffer = max(0, top + interior_buffer_size)
        right_interior_buffer = min(image.width, right - interior_buffer_size)
        bottom_interior_buffer = min(image.height, bottom - interior_buffer_size)
        draw.rectangle([(left_interior_buffer, top_interior_buffer), (right_interior_buffer, bottom_interior_buffer)], outline='green')

    # Save the image with bounding boxes and buffers
    image.save(output_path)
    image.show()

# Example usage:
images_directory = 'E:\\sample\\img'
labels_directory = 'E:\\sample\\label'

image_files = os.listdir(images_directory)
random_image_file = random.choice(image_files)
image_path = os.path.join(images_directory, random_image_file)

label_file = os.path.splitext(random_image_file)[0] + '.txt'
label_path = os.path.join(labels_directory, label_file)

# Create and visualize the exterior and interior buffers
output_buffered_image_path = 'output_buffered_image.jpg'
create_neighbor_8_buffers(image_path, label_path, output_buffered_image_path, exterior_buffer_size=5, interior_buffer_size=5)


In [ ]:
#average brightness of donut region only-- may be an error though
import os
import random
import numpy as np
from PIL import Image, ImageChops

def calculate_donut_statistics(image_path, label_path, exterior_buffer_size=5, interior_buffer_size=2):
    # Load the image
    image = Image.open(image_path)

    # Read the label file
    with open(label_path, 'r') as f:
        lines = f.readlines()

    # Initialize arrays for pixel values in the donut region
    donut_pixels = []

    # Iterate through bounding boxes and calculate donut region pixels
    for line in lines:
        class_id, x, y, width, height = map(float, line.strip().split(' '))
        left = int((x - width / 2) * image.width)
        top = int((y - height / 2) * image.height)
        right = int((x + width / 2) * image.width)
        bottom = int((y + height / 2) * image.height)

        # Calculate exterior buffer coordinates
        left_exterior_buffer = max(0, left - exterior_buffer_size)
        top_exterior_buffer = max(0, top - exterior_buffer_size)
        right_exterior_buffer = min(image.width, right + exterior_buffer_size)
        bottom_exterior_buffer = min(image.height, bottom + exterior_buffer_size)

        # Calculate interior buffer coordinates
        left_interior_buffer = max(0, left + interior_buffer_size)
        top_interior_buffer = max(0, top + interior_buffer_size)
        right_interior_buffer = min(image.width, right - interior_buffer_size)
        bottom_interior_buffer = min(image.height, bottom - interior_buffer_size)

        # Extract the donut region pixels
        donut_region = image.crop((left_exterior_buffer, top_exterior_buffer, right_exterior_buffer, bottom_exterior_buffer))
        interior_region = image.crop((left_interior_buffer, top_interior_buffer, right_interior_buffer, bottom_interior_buffer))

        # Subtract interior region from the donut region
        donut_region = ImageChops.difference(donut_region, interior_region)

        # Convert to NumPy array for pixel value calculation
        donut_pixels.extend(np.array(donut_region).reshape(-1, 3))

    # Calculate the average pixel brightness value
    if donut_pixels:
        average_brightness = np.mean(donut_pixels, axis=0)
        print("Average Brightness (R, G, B):", average_brightness)
    else:
        print("No pixels in the donut region.")

# Example usage:
images_directory = 'E:\\sample\\img'
labels_directory = 'E:\\sample\\label'

image_files = os.listdir(images_directory)
random_image_file = random.choice(image_files)
image_path = os.path.join(images_directory, random_image_file)

label_file = os.path.splitext(random_image_file)[0] + '.txt'
label_path = os.path.join(labels_directory, label_file)

# Calculate and print average pixel brightness for the donut region
calculate_donut_statistics(image_path, label_path, exterior_buffer_size=5, interior_buffer_size=2)


In [ ]:
#Per bounding box, calculates the average brightness between it and the rest of the background
import os
import random
import numpy as np
from PIL import Image, ImageDraw, ImageChops

def calculate_brightness(image, region):
    # Calculate the average brightness for the given region
    pixels = np.array(region).reshape(-1, 3)
    return np.mean(pixels, axis=0)

def calculate_statistics(image_path, label_path, exterior_buffer_size=5, interior_buffer_size=2):
    # Load the image
    image = Image.open(image_path)

    # Read the label file
    with open(label_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        class_id, x, y, width, height = map(float, line.strip().split(' '))
        left = int((x - width / 2) * image.width)
        top = int((y - height / 2) * image.height)
        right = int((x + width / 2) * image.width)
        bottom = int((y + height / 2) * image.height)

        # Calculate exterior buffer coordinates
        left_exterior_buffer = max(0, left - exterior_buffer_size)
        top_exterior_buffer = max(0, top - exterior_buffer_size)
        right_exterior_buffer = min(image.width, right + exterior_buffer_size)
        bottom_exterior_buffer = min(image.height, bottom + exterior_buffer_size)

        # Calculate interior buffer coordinates
        left_interior_buffer = max(0, left + interior_buffer_size)
        top_interior_buffer = max(0, top + interior_buffer_size)
        right_interior_buffer = min(image.width, right - interior_buffer_size)
        bottom_interior_buffer = min(image.height, bottom - interior_buffer_size)

        # Extract the donut region
        donut_region = image.crop((left_exterior_buffer, top_exterior_buffer, right_exterior_buffer, bottom_exterior_buffer))
        interior_region = image.crop((left_interior_buffer, top_interior_buffer, right_interior_buffer, bottom_interior_buffer))

        # Subtract interior region from the donut region to get the exterior region
        exterior_region = ImageChops.difference(donut_region, interior_region)

        # Calculate average brightness for each region
        average_brightness_interior = calculate_brightness(image, interior_region)
        average_brightness_exterior = calculate_brightness(image, exterior_region)

        print(f"Class {int(class_id)} - Interior Avg Brightness (R, G, B): {average_brightness_interior}")
        print(f"Class {int(class_id)} - Exterior Avg Brightness (R, G, B): {average_brightness_exterior}")

# Example usage:
images_directory = 'E:\\sample\\img'
labels_directory = 'E:\\sample\\label'

image_files = os.listdir(images_directory)
random_image_file = random.choice(image_files)
image_path = os.path.join(images_directory, random_image_file)

label_file = os.path.splitext(random_image_file)[0] + '.txt'
label_path = os.path.join(labels_directory, label_file)

# Calculate and print average pixel brightness for interior, exterior, and donut regions
calculate_statistics(image_path, label_path, exterior_buffer_size=5, interior_buffer_size=2)


In [ ]:
#For entire image-- calculates average brightness inside bbox, in the donut buffer, and for the background

import os
import random
import numpy as np
from PIL import Image, ImageDraw, ImageChops

def calculate_brightness(image, region):
    # Calculate the average brightness for the given region
    pixels = np.array(region).reshape(-1, 3)
    return np.mean(pixels, axis=0)

def calculate_image_statistics(image_path, label_path, exterior_buffer_size=5, interior_buffer_size=2):
    # Load the image
    image = Image.open(image_path)

    # Read the label file
    with open(label_path, 'r') as f:
        lines = f.readlines()

    # Initialize arrays for pixel values in different regions
    interior_pixels = []
    donut_pixels = []
    outside_pixels = []

    for line in lines:
        class_id, x, y, width, height = map(float, line.strip().split(' '))
        left = int((x - width / 2) * image.width)
        top = int((y - height / 2) * image.height)
        right = int((x + width / 2) * image.width)
        bottom = int((y + height / 2) * image.height)

        # Calculate exterior buffer coordinates
        left_exterior_buffer = max(0, left - exterior_buffer_size)
        top_exterior_buffer = max(0, top - exterior_buffer_size)
        right_exterior_buffer = min(image.width, right + exterior_buffer_size)
        bottom_exterior_buffer = min(image.height, bottom + exterior_buffer_size)

        # Calculate interior buffer coordinates
        left_interior_buffer = max(0, left + interior_buffer_size)
        top_interior_buffer = max(0, top + interior_buffer_size)
        right_interior_buffer = min(image.width, right - interior_buffer_size)
        bottom_interior_buffer = min(image.height, bottom - interior_buffer_size)

        # Extract the donut region
        donut_region = image.crop((left_exterior_buffer, top_exterior_buffer, right_exterior_buffer, bottom_exterior_buffer))
        interior_region = image.crop((left_interior_buffer, top_interior_buffer, right_interior_buffer, bottom_interior_buffer))

        # Subtract interior region from the donut region to get the exterior region
        exterior_region = ImageChops.difference(donut_region, interior_region)

        # Calculate average brightness for each region
        interior_pixels.extend(np.array(interior_region).reshape(-1, 3))
        donut_pixels.extend(np.array(donut_region).reshape(-1, 3))
        outside_pixels.extend(np.array(exterior_region).reshape(-1, 3))

    # Calculate the average pixel brightness for each region
    if interior_pixels:
        average_brightness_interior = np.mean(interior_pixels, axis=0)
        print("Average Brightness Inside Bounding Boxes (R, G, B):", average_brightness_interior)
    else:
        print("No pixels inside bounding boxes.")

    if donut_pixels:
        average_brightness_donut = np.mean(donut_pixels, axis=0)
        print("Average Brightness in Donut Regions (R, G, B):", average_brightness_donut)
    else:
        print("No pixels in donut regions.")

    if outside_pixels:
        average_brightness_outside = np.mean(outside_pixels, axis=0)
        print("Average Brightness Outside Donuts and Bounding Boxes (R, G, B):", average_brightness_outside)
    else:
        print("No pixels outside donuts and bounding boxes.")

# Example usage:
images_directory = 'E:\\sample\\img'
labels_directory = 'E:\\sample\\label'

image_files = os.listdir(images_directory)
random_image_file = random.choice(image_files)
image_path = os.path.join(images_directory, random_image_file)

label_file = os.path.splitext(random_image_file)[0] + '.txt'
label_path = os.path.join(labels_directory, label_file)

# Calculate and print average pixel brightness for different regions
calculate_image_statistics(image_path, label_path, exterior_buffer_size=5, interior_buffer_size=2)
